In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append("/n/home12/binxuwang/Github/vision")

In [2]:
from brainscore_vision import load_model
model = load_model("ReAlnet01_cornet")
model.start_task(...)
model.start_recording(...)
model.look_at(...)

ModuleNotFoundError: No module named 'brainio'

### RegNet 

### ReAlnet

https://github.com/ZitongLu1996/ReAlnet/

In [3]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import sys
sys.path.append("/n/home12/binxuwang/Github/Closed-loop-visual-insilico")

### Imagenet trained version

In [ ]:
from core.ReAlNet import CORnet_S, Encoder

In [5]:
torch.set_default_dtype(torch.float32)

transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# this cornet will be used for getting imagenet-based outputs as the classification targets
cornet = CORnet_S().to(device)
cornet = torch.nn.DataParallel(cornet)
url = f'https://s3.amazonaws.com/cornet-models/cornet_s-1d3f7974.pth'
ckpt_data = torch.utils.model_zoo.load_url(url)
cornet.load_state_dict(ckpt_data['state_dict'])
# this FAnet is what we are going to train
realnet = CORnet_S().to(device)
realnet = torch.nn.DataParallel(realnet)
url = f'https://s3.amazonaws.com/cornet-models/cornet_s-1d3f7974.pth'
ckpt_data = torch.utils.model_zoo.load_url(url)
realnet.load_state_dict(ckpt_data['state_dict'])

Downloading: "https://s3.amazonaws.com/cornet-models/cornet_s-1d3f7974.pth" to /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/torch_cache/hub/checkpoints/cornet_s-1d3f7974.pth
100%|██████████| 408M/408M [00:09<00:00, 45.6MB/s] 


<All keys matched successfully>

https://github.com/ZitongLu1996/ReAlnet

Where to get the final trained weights?

https://github.com/brain-score/vision/blob/37e435c8f9ecfc83d3e260575550eac29867ac4a/brainscore_vision/models/ReAlnet/model.py#L210

Here to load final weights and here to see the entry

https://github.com/brain-score/vision/blob/37e435c8f9ecfc83d3e260575550eac29867ac4a/brainscore_vision/models/ReAlnet/model.py

In [ ]:
# def model_load_weights(encoder, identifier: str):
#     # Download weights (Brain-Score team modification)
#     # Read the version id and sha1 from json file called "weights.json"
#     # weights_info = 
#     weights_info = {
#         "version_ids": {
#         "ReAlnet01": "75oY3CnI17U5S1f_yrZxl1XGhRfJEG9N",
#         "ReAlnet02": "TfGdm1CphJJ1vvkJGcm3n266PHvTuOaV",
#         "ReAlnet03": "dmohrH_AHZzgL_o8Xd2SDp6XCnjPOdAu",
#         "ReAlnet04": "45qJFXHihmIHdpHbjKWZco6STH1eh49p",
#         "ReAlnet05": "nqvoYgiBTyWSskjnpF9YOK4yYQfOnc_H",
#         "ReAlnet06": "6.cloFvnMihiicwQ0jkag8reEe4bVlxZ",
#         "ReAlnet07": "WKJaiN4b1ttpbGYNn8yVjng4LjCqWdk.",
#         "ReAlnet08": "vmouew6ePkPnKP.We8VnVxU7TifuhL.x",
#         "ReAlnet09": "53gqQ2tgS.5MEoncipy9mrBEqCc5izw5",
#         "ReAlnet10": "ZZFMhTm9KQYEXl8OGwKmnTr0S.pxkU0J"
#         },
#         "sha1s": {
#         "ReAlnet01": "05e4e401e8734b97e561aad306fc584b7e027225",
#         "ReAlnet02": "e85769fadb3c09ff88a7d73b01451b6bcccefd77",
#         "ReAlnet03": "f32d01d73380374ae501a1504e9c8cd219e9f0bf",
#         "ReAlnet04": "8062373fd6a74c52360420619235590d3688b4df",
#         "ReAlnet05": "88ca110f6b6d225b7b4e7dca02d2e7a906f5a8ed",
#         "ReAlnet06": "a1658c15a3c9d61262f87349c9fb7aa63854ac5b",
#         "ReAlnet07": "6a1c260839c75f6e6c018e06830562cdcda877e5",
#         "ReAlnet08": "1772211b27dd3a7d9255ac59d5f9b7e7cb6c3314",
#         "ReAlnet09": "159d96f0433a87c7063259dac4527325a3c7b79a",
#         "ReAlnet10": "dbdeaee9280267613ebce92dd5d515d89b544352"
#         }
#     }

#     version_id = weights_info['version_ids'][identifier]
#     sha1 = weights_info['sha1s'][identifier]

#     weights_path = load_weight_file(bucket="brainscore-storage", folder_name="brainscore-vision/models",
#                                     relative_path=f"ReAlnet/{identifier}_best_model_params.pt",
#                                     version_id=version_id,
#                                     sha1=sha1)

#     # Load weights onto CPU and remove "module." from keys
#     weights = torch.load(weights_path, map_location='cpu')
#     new_state_dict = {}
#     for key, val in weights.items():
#         # remove "module." (if it exists) from the key
#         new_key = key.replace("module.", "")
#         new_state_dict[new_key] = val

#     encoder.load_state_dict(new_state_dict)

#     # Retrieve the realnet portion from the encoder
#     realnet = encoder.realnet
#     realnet.eval()
#     return realnet


# # Construct CORnet_S
# realnet = CORnet_S()
# # (Optional) remove DataParallel if not needed for CPU
# # realnet = torch.nn.DataParallel(realnet)
# # Build encoder model
# encoder = Encoder(realnet, 340)
# # def get_model(identifier: str):
# model = model_load_weights(encoder, "ReAlnet01")
#     # preprocessing = functools.partial(load_preprocess_images, image_size=224)
#     # wrapper = PytorchWrapper(identifier=identifier, model=model, preprocessing=preprocessing)
#     # wrapper.image_size = 224
#     # return wrapper

In [12]:
import boto3
from pathlib import Path

def download_from_s3(
    bucket: str,
    key: str,
    local_path: Path,
    version_id: str = None,
):
    """
    Downloads an S3 object to local_path (creating parents if needed),
    optionally at a specific version.
    """
    s3 = boto3.client("s3")
    # ensure local directory exists
    local_path.parent.mkdir(parents=True, exist_ok=True)

    extra_args = {}
    if version_id:
        extra_args["VersionId"] = version_id

    s3.download_file(
        Bucket=bucket,
        Key=key,
        Filename=str(local_path),
        ExtraArgs=extra_args  # only passes VersionId if given
    )

weights_info = {
        "version_ids": {
        "ReAlnet01": "75oY3CnI17U5S1f_yrZxl1XGhRfJEG9N",
        "ReAlnet02": "TfGdm1CphJJ1vvkJGcm3n266PHvTuOaV",
        "ReAlnet03": "dmohrH_AHZzgL_o8Xd2SDp6XCnjPOdAu",
        "ReAlnet04": "45qJFXHihmIHdpHbjKWZco6STH1eh49p",
        "ReAlnet05": "nqvoYgiBTyWSskjnpF9YOK4yYQfOnc_H",
        "ReAlnet06": "6.cloFvnMihiicwQ0jkag8reEe4bVlxZ",
        "ReAlnet07": "WKJaiN4b1ttpbGYNn8yVjng4LjCqWdk.",
        "ReAlnet08": "vmouew6ePkPnKP.We8VnVxU7TifuhL.x",
        "ReAlnet09": "53gqQ2tgS.5MEoncipy9mrBEqCc5izw5",
        "ReAlnet10": "ZZFMhTm9KQYEXl8OGwKmnTr0S.pxkU0J"
        },
        "sha1s": {
        "ReAlnet01": "05e4e401e8734b97e561aad306fc584b7e027225",
        "ReAlnet02": "e85769fadb3c09ff88a7d73b01451b6bcccefd77",
        "ReAlnet03": "f32d01d73380374ae501a1504e9c8cd219e9f0bf",
        "ReAlnet04": "8062373fd6a74c52360420619235590d3688b4df",
        "ReAlnet05": "88ca110f6b6d225b7b4e7dca02d2e7a906f5a8ed",
        "ReAlnet06": "a1658c15a3c9d61262f87349c9fb7aa63854ac5b",
        "ReAlnet07": "6a1c260839c75f6e6c018e06830562cdcda877e5",
        "ReAlnet08": "1772211b27dd3a7d9255ac59d5f9b7e7cb6c3314",
        "ReAlnet09": "159d96f0433a87c7063259dac4527325a3c7b79a",
        "ReAlnet10": "dbdeaee9280267613ebce92dd5d515d89b544352"
        }
    }
identifier = "ReAlnet01"
version_id = weights_info['version_ids'][identifier]
sha1 = weights_info['sha1s'][identifier]
# original loading method, we use our own s3 method. 
# weights_path = load_weight_file(bucket="brainscore-storage", folder_name="brainscore-vision/models",
#                             relative_path=f"ReAlnet/{identifier}_best_model_params.pt",
#                             version_id=version_id,
#                             sha1=sha1)
bucket = "brainscore-storage"
key = f"brainscore-vision/models/ReAlnet/{identifier}_best_model_params.pt"
version_id = version_id  # or None
save_root = f"/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/brain_score_cache"
local_file = Path(f"{save_root}/{identifier}_best_model_params.pt")
download_from_s3(bucket="brainscore-storage", key=key, local_path=local_file, version_id=version_id)
data = torch.load(local_file, map_location="cpu")

In [17]:
import boto3
from pathlib import Path

def download_from_s3(
    bucket: str,
    key: str,
    local_path: Path,
    version_id: str = None,
):
    """
    Downloads an S3 object to local_path (creating parents if needed),
    optionally at a specific version.
    """
    s3 = boto3.client("s3")
    # ensure local directory exists
    local_path.parent.mkdir(parents=True, exist_ok=True)

    extra_args = {}
    if version_id:
        extra_args["VersionId"] = version_id

    s3.download_file(
        Bucket=bucket,
        Key=key,
        Filename=str(local_path),
        ExtraArgs=extra_args  # only passes VersionId if given
    )


weights_info = {
        "version_ids": {
        "ReAlnet01": "75oY3CnI17U5S1f_yrZxl1XGhRfJEG9N",
        "ReAlnet02": "TfGdm1CphJJ1vvkJGcm3n266PHvTuOaV",
        "ReAlnet03": "dmohrH_AHZzgL_o8Xd2SDp6XCnjPOdAu",
        "ReAlnet04": "45qJFXHihmIHdpHbjKWZco6STH1eh49p",
        "ReAlnet05": "nqvoYgiBTyWSskjnpF9YOK4yYQfOnc_H",
        "ReAlnet06": "6.cloFvnMihiicwQ0jkag8reEe4bVlxZ",
        "ReAlnet07": "WKJaiN4b1ttpbGYNn8yVjng4LjCqWdk.",
        "ReAlnet08": "vmouew6ePkPnKP.We8VnVxU7TifuhL.x",
        "ReAlnet09": "53gqQ2tgS.5MEoncipy9mrBEqCc5izw5",
        "ReAlnet10": "ZZFMhTm9KQYEXl8OGwKmnTr0S.pxkU0J"
        },
        "sha1s": {
        "ReAlnet01": "05e4e401e8734b97e561aad306fc584b7e027225",
        "ReAlnet02": "e85769fadb3c09ff88a7d73b01451b6bcccefd77",
        "ReAlnet03": "f32d01d73380374ae501a1504e9c8cd219e9f0bf",
        "ReAlnet04": "8062373fd6a74c52360420619235590d3688b4df",
        "ReAlnet05": "88ca110f6b6d225b7b4e7dca02d2e7a906f5a8ed",
        "ReAlnet06": "a1658c15a3c9d61262f87349c9fb7aa63854ac5b",
        "ReAlnet07": "6a1c260839c75f6e6c018e06830562cdcda877e5",
        "ReAlnet08": "1772211b27dd3a7d9255ac59d5f9b7e7cb6c3314",
        "ReAlnet09": "159d96f0433a87c7063259dac4527325a3c7b79a",
        "ReAlnet10": "dbdeaee9280267613ebce92dd5d515d89b544352"
        }
    }

def build_model(identifier: str):
    save_root = f"/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/brain_score_cache"
    local_weights_path = f"{save_root}/{identifier}_best_model_params.pt"
    realnet = CORnet_S()
    # (Optional) remove DataParallel if not needed for CPU
    # realnet = torch.nn.DataParallel(realnet)
    # Build encoder model
    encoder = Encoder(realnet, 340)
    version_id = weights_info['version_ids'][identifier]
    sha1 = weights_info['sha1s'][identifier]
    if not os.path.exists(local_weights_path):
        key = f"brainscore-vision/models/ReAlnet/{identifier}_best_model_params.pt"
        download_from_s3(bucket="brainscore-storage", key=key, 
                         local_path=Path(local_weights_path), version_id=version_id)
    
    weights_data = torch.load(local_weights_path, map_location="cpu")
    new_state_dict = {}
    for key, val in weights_data.items():
        # remove "module." (if it exists) from the key
        new_key = key.replace("module.", "")
        new_state_dict[new_key] = val

    encoder.load_state_dict(new_state_dict)
    # Retrieve the realnet portion from the encoder
    realnet = encoder.realnet
    realnet.eval()
    realnet.requires_grad_(False)
    return realnet

In [19]:
build_model("ReAlnet01");

### AlexNet

In [25]:
from core.alexnet_torch import alexnet_v2_pytorch
import os

In [28]:
def build_alexnet_brainscore(identifier: str = "training_seed_01"):
    model = alexnet_v2_pytorch()
    local_weights_path = f"{save_root}/alexnet_weights_{identifier}.pth"
    if not os.path.exists(local_weights_path):
        # sha1 = "4b1bb7810d5288631c04cf4cde882540d6ebee77"
        key = f"models/model_weights/{identifier}.pth"
        download_from_s3(bucket="brainscorevariability", key=key, 
                        local_path=Path(local_weights_path), version_id=None)
    model_weights = torch.load(local_weights_path, map_location='cpu') 
    model.load_state_dict(model_weights)
    model.eval()
    model.requires_grad_(False)
    return model

transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

In [30]:
build_alexnet_brainscore("training_seed_01")

AlexNetV2(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Conv2d(256, 4096, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Conv2d(4096, 4096, kernel_size=

### Regnet

In [42]:
import timm

In [43]:
model = timm.create_model(
    'regnety_640.seer',
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
)
model = model.eval()
# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [44]:
model

RegNet(
  (stem): ConvNormAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (s1): RegStage(
    (b1): Bottleneck(
      (conv1): ConvNormAct(
        (conv): Conv2d(32, 328, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNormAct2d(
          328, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
      )
      (conv2): ConvNormAct(
        (conv): Conv2d(328, 328, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNormAct2d(
          328, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
      )
      (se): SEModule(
        (fc1): Conv2d(328, 8, kernel_si

### Hooking experiment

In [2]:
from circuit_toolkit.layer_hook_utils import featureFetcher, recursive_print
from circuit_toolkit.dataset_utils import ImagePathDataset

/n/home12/binxuwang/Github/circuit_toolkit/circuit_toolkit/GAN_invert_utils.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange, tqdm


In [1]:
import sys
sys.path.append("/n/home12/binxuwang/Github/Closed-loop-visual-insilico")
from core.model_load_utils import load_model_transform

In [3]:
realnet, preprocess = load_model_transform("ReAlnet01")

/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
alexnet, preprocess = load_model_transform("AlexNet_training_seed_01")

In [5]:
fetcher = featureFetcher(alexnet, print_module=False, )

In [7]:
fetcher.module_names

OrderedDict([('0', 'Image'),
             ('1', '.features.Conv2d0'),
             ('2', '.features.ReLU1'),
             ('3', '.features.MaxPool2d2'),
             ('4', '.features.Conv2d3'),
             ('5', '.features.ReLU4'),
             ('6', '.features.MaxPool2d5'),
             ('7', '.features.Conv2d6'),
             ('8', '.features.ReLU7'),
             ('9', '.features.Conv2d8'),
             ('10', '.features.ReLU9'),
             ('11', '.features.Conv2d10'),
             ('12', '.features.ReLU11'),
             ('13', '.features.MaxPool2d12'),
             ('14', '.features'),
             ('15', '.classifier.Conv2d0'),
             ('16', '.classifier.ReLU1'),
             ('17', '.classifier.Dropout2'),
             ('18', '.classifier.Conv2d3'),
             ('19', '.classifier.ReLU4'),
             ('20', '.classifier.Dropout5'),
             ('21', '.classifier.Conv2d6'),
             ('22', '.classifier'),
             ('23', '.AlexNetV2')])

In [40]:
fetcher2 = featureFetcher(realnet, print_module=False, )

In [9]:
recursive_print(alexnet, deepest=2)

[AlexNetV2]
(features): Sequential
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
(classifier): Sequential
  (0): Conv2d(256, 4096, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Conv2d(4096, 4096, kernel_size=(1, 1), stride=(1, 1))
  (4): ReLU(inplace=

In [8]:
recursive_print(realnet, deepest=2)

[Sequential]
(V1): Sequential
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (nonlin1): ReLU(inplace=True)
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (nonlin2): ReLU(inplace=True)
  (output): Identity()
(V2): CORblock_S
  (conv_input): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (skip): Conv2d(128, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
  (norm_skip): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (nonlin1): ReLU(inplace=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), pad

In [41]:
fetcher2.module_names

OrderedDict([('0', 'Image'),
             ('1', '.V1.Conv2dconv1'),
             ('2', '.V1.BatchNorm2dnorm1'),
             ('3', '.V1.ReLUnonlin1'),
             ('4', '.V1.MaxPool2dpool'),
             ('5', '.V1.Conv2dconv2'),
             ('6', '.V1.BatchNorm2dnorm2'),
             ('7', '.V1.ReLUnonlin2'),
             ('8', '.V1.Identityoutput'),
             ('9', '.V1'),
             ('10', '.V2.Conv2dconv_input'),
             ('11', '.V2.Conv2dskip'),
             ('12', '.V2.BatchNorm2dnorm_skip'),
             ('13', '.V2.Conv2dconv1'),
             ('14', '.V2.BatchNorm2dnorm1_0'),
             ('15', '.V2.ReLUnonlin1'),
             ('16', '.V2.Conv2dconv2'),
             ('17', '.V2.BatchNorm2dnorm2_0'),
             ('18', '.V2.ReLUnonlin2'),
             ('19', '.V2.Conv2dconv3'),
             ('20', '.V2.BatchNorm2dnorm3_0'),
             ('21', '.V2.ReLUnonlin3'),
             ('22', '.V2.Identityoutput'),
             ('23', '.V2.Conv2dconv1'),
             ('24', 

In [51]:
recursive_print(model, deepest=2)

[RegNet]
(stem): ConvNormAct
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNormAct2d
(s1): RegStage
  (b1): Bottleneck
  (b2): Bottleneck
(s2): RegStage
  (b1): Bottleneck
  (b2): Bottleneck
  (b3): Bottleneck
  (b4): Bottleneck
  (b5): Bottleneck
(s3): RegStage
  (b1): Bottleneck
  (b2): Bottleneck
  (b3): Bottleneck
  (b4): Bottleneck
  (b5): Bottleneck
  (b6): Bottleneck
  (b7): Bottleneck
  (b8): Bottleneck
  (b9): Bottleneck
  (b10): Bottleneck
  (b11): Bottleneck
  (b12): Bottleneck
(s4): RegStage
  (b1): Bottleneck
(final_conv): Identity()
(head): ClassifierHead
  (global_pool): SelectAdaptivePool2d
  (drop): Dropout(p=0.0, inplace=False)
  (fc): Identity()
  (flatten): Identity()


In [45]:
fetcher3 = featureFetcher(model, print_module=False, )
fetcher3.module_names

OrderedDict([('0', 'Image'),
             ('1', '.stem.Conv2dconv'),
             ('2', '.stem.bn.Identitydrop'),
             ('3', '.stem.bn.ReLUact'),
             ('4', '.stem.BatchNormAct2dbn'),
             ('5', '.ConvNormActstem'),
             ('6', '.s1.b1.conv1.Conv2dconv'),
             ('7', '.s1.b1.conv1.bn.Identitydrop'),
             ('8', '.s1.b1.conv1.bn.ReLUact'),
             ('9', '.s1.b1.conv1.BatchNormAct2dbn'),
             ('10', '.s1.b1.ConvNormActconv1'),
             ('11', '.s1.b1.conv2.Conv2dconv'),
             ('12', '.s1.b1.conv2.bn.Identitydrop'),
             ('13', '.s1.b1.conv2.bn.ReLUact'),
             ('14', '.s1.b1.conv2.BatchNormAct2dbn'),
             ('15', '.s1.b1.ConvNormActconv2'),
             ('16', '.s1.b1.se.Conv2dfc1'),
             ('17', '.s1.b1.se.Identitybn'),
             ('18', '.s1.b1.se.ReLUact'),
             ('19', '.s1.b1.se.Conv2dfc2'),
             ('20', '.s1.b1.se.Sigmoidgate'),
             ('21', '.s1.b1.SEModulese'),

In [55]:
[name for i, name in fetcher3.module_names.items() if "Bottleneck" in name or "RegStage" in name]

['.s1.Bottleneckb1',
 '.s1.Bottleneckb2',
 '.RegStages1',
 '.s2.Bottleneckb1',
 '.s2.Bottleneckb2',
 '.s2.Bottleneckb3',
 '.s2.Bottleneckb4',
 '.s2.Bottleneckb5',
 '.RegStages2',
 '.s3.Bottleneckb1',
 '.s3.Bottleneckb2',
 '.s3.Bottleneckb3',
 '.s3.Bottleneckb4',
 '.s3.Bottleneckb5',
 '.s3.Bottleneckb6',
 '.s3.Bottleneckb7',
 '.s3.Bottleneckb8',
 '.s3.Bottleneckb9',
 '.s3.Bottleneckb10',
 '.s3.Bottleneckb11',
 '.s3.Bottleneckb12',
 '.RegStages3',
 '.s4.Bottleneckb1',
 '.RegStages4']